In [1]:
from fuzzywuzzy import fuzz
import datetime
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from airflow.providers.mysql.hooks.mysql import MySqlHook
from airflow.providers.postgres.hooks.postgres import PostgresHook

warehouse_hook = PostgresHook(postgres_conn_id='rds_afsg_ds_prod_postgresql_dwh', schema='afsg_ds_prod_postgresql_dwh')
mifos_hook = MySqlHook(mysql_conn_id='mifos_db', schema='mifostenant-safaricom')
mifos1_hook = MySqlHook(mysql_conn_id='mifos_db', schema='mifostenant-default')
airflow_hook = MySqlHook(mysql_conn_id='airflow_server_admin', schema='monitoring')
middleware_hook = PostgresHook(postgres_conn_id='asante_mfs_middleware', schema='asante_datawarehouse')
jubilee_hook = MySqlHook(mysql_conn_id='mifos_db', schema='mifostenant-jubilee')

/tmp/ipykernel_2925/3000044281.py:10 DeprecationWarning: The "schema" arg has been renamed to "database" as it contained the database name.Please use "database" to set the database name.

/tmp/ipykernel_2925/3000044281.py:14 DeprecationWarning: The "schema" arg has been renamed to "database" as it contained the database name.Please use "database" to set the database name.

#### mobile number updates

In [3]:
dt = pd.read_excel('~/data/Mobile Numbers Updates.xlsx')

In [4]:
print(dt.shape)
dt.rename(columns={'mobile_number': 'updated_mobile_number'}, inplace=True)
dt.head()

(98, 3)


,Unnamed: 0,store_number,updated_mobile_number
0,1,7517465,254706314291
1,5,7988250,254700652092
2,6,7138079,254722996391
3,7,7036690,254727802295
4,8,7953755,254729671885


In [ ]:
dt['store_number'] = dt['store_number'].apply(
    lambda x: str(x).replace('.0', '').strip()
)
warehouse_clients = warehouse_hook.get_pandas_df(
    sql="""
        select mobile_number as current_mobile_number, store_number from bloomlive.client_materialized_summary_view
        where store_number in %(sn)s
    """,
    parameters={'sn': tuple(dt['store_number'].tolist())}
)
mrgd = dt.merge(
    warehouse_clients,
    left_on='store_number',
    right_on='store_number',
    how='left'
)

In [5]:
mrgd[['store_number', 'current_mobile_number', 'updated_mobile_number']]

NameError: name 'mrgd' is not defined

In [ ]:
def store_mobile_number_updates(updated: list, source: str):
    import pytz

    airflow_hook = MySqlHook(mysql_conn_id='airflow_server_admin', schema='bloom_pipeline')

    update_logs = pd.DataFrame(
        [result for results in updated for result in results],
        columns=['updated_on_timestamp', 'current_mobile_number', 'previous_mobile_number', 'client_surrogate_id']
    )
    update_logs['source'] = source

    utc_timezone = pytz.timezone('UTC')
    eat_timezone = pytz.timezone('Africa/Nairobi')

    update_logs['updated_on_timestamp'] = update_logs['updated_on_timestamp'].apply(
        lambda x: utc_timezone.localize(
            datetime.datetime.strptime(x.split("+")[0], '%Y-%m-%d %H:%M:%S.%f')).astimezone(eat_timezone)
    )
    airflow_hook.insert_rows(
        table='mobile_number_updates',
        target_fields=update_logs.reindex(sorted(update_logs.index)).columns.tolist(),
        rows=tuple(update_logs.reindex(sorted(update_logs.index)).replace({np.NAN: None}).itertuples(index=False,
                                                                                                     name=None)),
        commit_every=100
    )

In [ ]:
mobile_number_updates = []
for index, row in mrgd.replace({np.NAN: None}).iterrows():
    mobile_number_updates.append(
        f"UPDATE bloomlive.client_dimension SET mobile_number={row['updated_mobile_number']} WHERE store_number='{row['store_number']}'  returning current_timestamp::varchar, '{row['updated_mobile_number']}', '{row['current_mobile_number']}', surrogate_id"
    )

In [ ]:
# if len(mobile_number_updates) > 0:
#     # Run the updates and store the results
#     updated = warehouse_hook.run(sql=mobile_number_updates, handler=lambda x: x.fetchall())
#     store_mobile_number_updates(updated=updated, source='list from Operations team')

#### National ID updates

In [3]:
dt2 = pd.read_excel('~/data/National IDs Updates.xlsx')

In [4]:
print(dt2.shape)
dt2.rename(columns={'national_id': 'updated_national_id'}, inplace=True)
dt2.head()

(26804, 3)


,Unnamed: 0,store_number,updated_national_id
0,0,6108816,23322127
1,1,7127165,32504894
2,3,6141509,20801490
3,4,7490909,35008603
4,5,7517465,22282737


In [ ]:
dt2['store_number'] = dt2['store_number'].apply(
    lambda x: str(x).replace('.0', '').strip()
)
warehouse_clients2 = warehouse_hook.get_pandas_df(
    sql="""
        select national_id as current_national_id, store_number, is_iprs_checked, is_iprs_validated from bloomlive.client_materialized_summary_view
        where store_number in %(sn)s
    """,
    parameters={'sn': tuple(dt2['store_number'].tolist())}
)
mrgd2 = dt2.merge(
    warehouse_clients2,
    left_on='store_number',
    right_on='store_number',
    how='left'
)

In [ ]:
def store_national_id_updates(updated: list, source: str):
    import pytz
    airflow_hook = MySqlHook(mysql_conn_id='airflow_server_admin', schema='bloom_pipeline')

    update_logs = pd.DataFrame(
        [result for results in updated for result in results],
        columns=['updated_on_timestamp', 'current_national_id', 'previous_national_id', 'client_surrogate_id']
    )
    update_logs['source'] = source

    utc_timezone = pytz.timezone('UTC')
    eat_timezone = pytz.timezone('Africa/Nairobi')

    update_logs['updated_on_timestamp'] = update_logs['updated_on_timestamp'].apply(
        lambda x: utc_timezone.localize(
            datetime.datetime.strptime(x.split("+")[0], '%Y-%m-%d %H:%M:%S.%f')).astimezone(eat_timezone)
    )
    airflow_hook.insert_rows(
        table='national_id_updates',
        target_fields=update_logs.reindex(sorted(update_logs.index)).columns.tolist(),
        rows=tuple(update_logs.reindex(sorted(update_logs.index)).replace({np.NAN: None}).itertuples(index=False,
                                                                                                     name=None)),
        commit_every=100
    )

In [ ]:
mrgd2[['store_number', 'current_national_id', 'updated_national_id', 'is_iprs_checked', 'is_iprs_validated']]

In [ ]:
mrgd2[mrgd2['is_iprs_checked']]

In [ ]:
mrgd2[mrgd2['is_iprs_validated']]

In [ ]:
not_iprs_checked = mrgd2[~mrgd2['is_iprs_checked']]
not_iprs_checked

In [ ]:
not_iprs_checked_updates = []
for index, row in not_iprs_checked.replace({np.NAN: "null"}).iterrows():
    not_iprs_checked_updates.append(
        f"UPDATE bloomlive.client_dimension set national_id = '{row['updated_national_id']}' where store_number = '{row['store_number']}' returning current_timestamp::varchar, '{row['updated_national_id']}', '{row['current_national_id']}', surrogate_id"
    )

In [ ]:
not_iprs_checked_updates

In [ ]:
# if len(not_iprs_checked_updates) > 0:
#     updated = warehouse_hook.run(sql=not_iprs_checked_updates, handler=lambda x: x.fetchall())
#     store_national_id_updates(updated=updated, source='List from Operations team')

### Fuzzy Match Company Names and Customer Names

In [5]:
ops_list = pd.read_excel('~/data/Bloom_KYC_Dump_20230321_v1.0.xlsx')

In [6]:
ops_list.head()

,mpsa_orga_shrt,cust_id_nmbr,enty_name,opt_in_date,mpsa_enty_phne,ds_cst_name
0,30,28676956,WILTER MORAA ARANDI,2022-06-09,254714319251,WILTER ARANDI
1,85,27841627,SAN GEORGE MEDICAL CENTRE,2022-01-07,254717294464,GEORGE NYAGA
2,1582,22221408,PUB CANAAN,2022-02-04,254745740738,OBADIAH KIMOTHO
3,1600,29062759,LEOLINO VENTURES WINES SPIRITS,2021-11-15,254711506097,LINUS OKWERO
4,1609,20862848,DAVID MURITHI PHILIP,2022-02-04,254721833709,DAVID MURITHI PHILIP


In [7]:
ops_list[ops_list['mpsa_orga_shrt'] == 279851]

,mpsa_orga_shrt,cust_id_nmbr,enty_name,opt_in_date,mpsa_enty_phne,ds_cst_name
1997,279851,20411464,DATLEAN LAUNDRY,2021-11-12,254722691217,CAROLYNE WAMBUI KAGWI


In [8]:
customers = warehouse_hook.get_pandas_df(
    sql="""
        select surrogate_id, bloom_version, store_number, national_id, first_name, middle_name, last_name, iprs_first_name, iprs_other_name, iprs_surname, submitted_on_date, company_name, is_iprs_validated from bloomlive.client_materialized_summary_view where national_id in %(id_nos)s
    """,
    parameters={'id_nos': tuple(ops_list['cust_id_nmbr'].astype(str).tolist())}
)

[2023-04-24 11:01:06,321] {base.py:73} INFO - Using connection ID 'airflow_warehouse_user' for task execution.


/home/henrykuria/airflow_venv/lib/python3.10/site-packages/pandas/io/sql.py:761 UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy

In [9]:
customers[~customers['is_iprs_validated']]

,surrogate_id,bloom_version,store_number,national_id,first_name,middle_name,last_name,iprs_first_name,iprs_other_name,iprs_surname,submitted_on_date,company_name,is_iprs_validated
367,194905,2.0,7656294,28296041,Peninah,Wanjiru,Maina,None,None,None,2022-07-07,PENINAH MAINA,False
1438,230393,2.0,7752852,33814054,Boniface,Mutisya,Musembi,None,None,None,2022-11-11,BONIFACE MUTISYA MUSEMBI,False
1980,229054,2.0,7102757,22967526,Dorcas,Rutto,,None,None,None,2022-11-04,DORCAS RUTTO,False
3078,224018,2.0,7985122,24877663,Dotty,Akoth,Kehara,None,None,None,2022-10-17,DOTTY AKOTH KEHARA,False
3263,193559,2.0,7666868,37105021,Virginia,Nyambura,Wambui,None,None,None,2022-06-30,VIRGINIA NYAMBURA WAMBUI,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
118972,164972,2.0,7024538,13795603,Elias,Irungu,Wanjiru,None,None,None,2022-04-11,ELIAS IRUNGU WANJIRU,False
119613,236658,2.0,6011098,26149822,Sharon,Osore,,None,None,None,2022-12-07,SHARON OSORE,False
120249,224517,2.0,7266068,23298253,Betty,C,Chelimo,None,None,None,2022-10-18,BETTY C CHELIMO,False
120541,222443,2.0,7877734,24301974,Wamutwa,Wanduu,,None,None,None,2022-10-12,WAMUTWA WANDUU,False


In [10]:
# merged = customers.merge(
#     ops_list[['mpsa_orga_shrt', 'cust_id_nmbr', 'enty_name', 'opt_in_date', 'ds_cst_name']].rename(
#         columns={'ds_cst_name': 'ds_company_name', 'enty_name': 'ds_customer_name'}
#     ).astype({'cust_id_nmbr': str}),
#     left_on='national_id',
#     right_on='cust_id_nmbr',
#     how='left'
# )
merged = customers.merge(
    ops_list[['mpsa_orga_shrt', 'cust_id_nmbr', 'enty_name', 'opt_in_date', 'ds_cst_name']].rename(
        columns={'enty_name': 'ds_company_name', 'ds_cst_name': 'ds_customer_name'}
    ).astype({'cust_id_nmbr': str}),
    left_on='national_id',
    right_on='cust_id_nmbr',
    how='left'
)

In [11]:
merged.head()

,surrogate_id,bloom_version,store_number,national_id,first_name,middle_name,last_name,iprs_first_name,iprs_other_name,iprs_surname,submitted_on_date,company_name,is_iprs_validated,mpsa_orga_shrt,cust_id_nmbr,ds_company_name,opt_in_date,ds_customer_name
0,149766,2.0,170329,32515103,Igantius,Kimindu,Maeke,Igantius,Kimindu,Maeke,2021-11-13,SAFE SPACE WINES AND SPIRIT,True,170329,32515103,IGNATIUS KIMINDU MAEKE,2022-12-23,SAFE SPACE WINES AND SPIRIT
1,146705,2.0,173275,26637201,Eric,Kirimi,Nkonge,Eric,Kirimi,Nkonge,2021-11-13,GEOFFE MITCH SERVICES,True,173275,26637201,ERIC KIRIMI NKONGE,2022-07-26,GEOFFE MITCH SERVICES
2,150260,2.0,200725,23024758,Jackson,Wamboo,Wangechi,Jackson,Wamboo,Wangechi,2021-11-15,JACKSON WAMBOO WANGECHI,True,200725,23024758,JACKSON WANGECHI WAMBOO,2022-06-25,JACKSON WAMBOO WANGECHI
3,149454,2.0,319435,25509730,Hellen,Bosibori,Nyatome,Hellen,Bosibori,Nyatome,2021-11-22,OBOMO PUB,True,319435,25509730,HELLEN BOSIBORI NYATOME,2022-05-18,OBOMO PUB
4,149454,2.0,319435,25509730,Hellen,Bosibori,Nyatome,Hellen,Bosibori,Nyatome,2021-11-22,OBOMO PUB,True,7917688,25509730,HELLEN BOSIBORI NYATOME,2022-09-21,OBOMO BAR AND RESTAURANT


In [12]:
merged['full_name_mifos'] = merged[['first_name', 'middle_name', 'last_name']].apply(
    lambda x: f"{x['first_name']} {x['middle_name']} {x['last_name']}",
    axis=1
)

In [13]:
merged[['first_name', 'middle_name', 'last_name', 'full_name_mifos']].head(n=2)

,first_name,middle_name,last_name,full_name_mifos
0,Igantius,Kimindu,Maeke,Igantius Kimindu Maeke
1,Eric,Kirimi,Nkonge,Eric Kirimi Nkonge


In [14]:
merged['full_name_iprs'] = merged[['iprs_first_name', 'iprs_other_name', 'iprs_surname']].apply(
    lambda x: f"{x['iprs_first_name']} {x['iprs_other_name']} {x['iprs_surname']}",
    axis=1
)

In [15]:
merged[['iprs_first_name', 'iprs_other_name', 'iprs_surname', 'full_name_iprs']].head(n=2)

,iprs_first_name,iprs_other_name,iprs_surname,full_name_iprs
0,Igantius,Kimindu,Maeke,Igantius Kimindu Maeke
1,Eric,Kirimi,Nkonge,Eric Kirimi Nkonge


In [16]:
# Split the full name by hyphen and store the resulting list in a new column
merged['ds_customer_name'] = merged['ds_customer_name'].apply(
    lambda x: str(x).lower().strip() if not pd.isnull(x) else x
)
merged['full_name_iprs'] = merged['full_name_iprs'].apply(
    lambda x: str(x).lower().strip() if not pd.isnull(x) else x
)
merged['full_name_mifos'] = merged['full_name_mifos'].apply(
    lambda x: str(x).lower().strip() if not pd.isnull(x) else x
)
merged['name_parts'] = merged['ds_customer_name'].str.split('-')

# If there are more than 3 name parts, join the last 3 into a single name part
if len(merged['name_parts'][0]) > 3:
    merged['name_parts'][0][2:] = [' '.join(merged['name_parts'][0][2:])]

# Join the first three name parts with a space separator to form the full name
merged['full_name_ops'] = merged['name_parts'].apply(lambda x: ' '.join(x[:3]) if type(x) is not float else 'None')


In [17]:
merged[['ds_customer_name', 'full_name_ops']].head(n=2)

,ds_customer_name,full_name_ops
0,safe space wines and spirit,safe space wines and spirit
1,geoffe mitch services,geoffe mitch services


In [18]:
merged["mifos_vs_ops_similarity_score"] = merged.apply(lambda row: fuzz.token_set_ratio(row['full_name_mifos'], row['full_name_ops']), axis=1)

merged["mifos_vs_iprs_similarity_score"] = merged.apply(lambda row: fuzz.token_set_ratio(row['full_name_mifos'], row['full_name_iprs']), axis=1)

merged["ops_vs_iprs_similarity_score"] = merged.apply(lambda row: fuzz.token_set_ratio(row['full_name_ops'], row['full_name_iprs']), axis=1)

In [19]:
final = merged[[
    'national_id', 'cust_id_nmbr', 'mpsa_orga_shrt', 'opt_in_date', 'full_name_ops', 'ops_vs_iprs_similarity_score', 'full_name_mifos', 'mifos_vs_ops_similarity_score', 'full_name_iprs', 'mifos_vs_iprs_similarity_score', 'is_iprs_validated']].rename(columns={
    'full_name_ops': 'Full Name (OPS)', 'full_name_mifos': 'Full Name (WAREHOUSE)',
    'full_name_iprs': 'Full Name (IPRS)','ops_vs_iprs_similarity_score': 'OPS-IPRS Score',
    'mifos_vs_ops_similarity_score': 'WAREHOUSE-OPS Score',
    'mifos_vs_iprs_similarity_score': 'WAREHOUSE-IPRS Score'
}).round(2)[['cust_id_nmbr', 'national_id', 'mpsa_orga_shrt', 'opt_in_date', 'Full Name (OPS)', 'Full Name (WAREHOUSE)', 'Full Name (IPRS)', 'OPS-IPRS Score', 'WAREHOUSE-OPS Score', 'WAREHOUSE-IPRS Score', 'is_iprs_validated']]
final.drop_duplicates(subset=['national_id', 'mpsa_orga_shrt', 'opt_in_date', 'Full Name (OPS)', 'Full Name (IPRS)', 'OPS-IPRS Score', 'is_iprs_validated'], inplace=True)
final['opt_in_rank'] = final.groupby('national_id')['opt_in_date'].rank(method='dense', ascending=False)
final.head()

,cust_id_nmbr,national_id,mpsa_orga_shrt,opt_in_date,Full Name (OPS),Full Name (WAREHOUSE),Full Name (IPRS),OPS-IPRS Score,WAREHOUSE-OPS Score,WAREHOUSE-IPRS Score,is_iprs_validated,opt_in_rank
0,32515103,32515103,170329,2022-12-23,safe space wines and spirit,igantius kimindu maeke,igantius kimindu maeke,33,33,100,True,1.0
1,26637201,26637201,173275,2022-07-26,geoffe mitch services,eric kirimi nkonge,eric kirimi nkonge,36,36,100,True,1.0
2,23024758,23024758,200725,2022-06-25,jackson wamboo wangechi,jackson wamboo wangechi,jackson wamboo wangechi,100,100,100,True,1.0
3,25509730,25509730,319435,2022-05-18,obomo pub,hellen bosibori nyatome,hellen bosibori nyatome,25,25,100,True,2.0
4,25509730,25509730,7917688,2022-09-21,obomo bar and restaurant,hellen bosibori nyatome,hellen bosibori nyatome,34,34,100,True,1.0


In [20]:
# final.to_csv('~/data/kyc_fuzzy_match_final.csv')
already_updated = pd.read_csv('/home/henrykuria/final_kyc_updates_2023-04-18 10:48:40.065555.csv')
final = final[~final['national_id'].isin(already_updated['national_id'].tolist())]
# final[final['mpsa_orga_shrt'] == 279851]

In [28]:
# Search if 'A' column contains a number
contains_number = final['Full Name (OPS)'].str.contains(r'\d+', na=False)

# Display the rows where 'A' column contains a number
final = final.loc[final[~contains_number].index]

In [29]:
# Final Dataset
# updates batch 1 - Ops and IPRS names are 100% matched 1457
# updates_batch_1 = final[
#     (final['opt_in_rank'] == 1) &
#     (final['WAREHOUSE-OPS Score'] < 78) &
#     (final['OPS-IPRS Score'] == 100)
# ].drop_duplicates(subset=['national_id'])
# updates_batch_1

updates_batch_1_ = final[
    (final['opt_in_rank'] == 1) &
    (final['WAREHOUSE-OPS Score'] < 78) &
    (final['OPS-IPRS Score'] == 100)
].drop_duplicates(subset=['national_id'])
updates_batch_1_

,cust_id_nmbr,national_id,mpsa_orga_shrt,opt_in_date,Full Name (OPS),Full Name (WAREHOUSE),Full Name (IPRS),OPS-IPRS Score,WAREHOUSE-OPS Score,WAREHOUSE-IPRS Score,is_iprs_validated,opt_in_rank
6,8633566,8633566,4011791,2022-09-16,jasan nyoike kariuki,jasan karuiki,jasan nyoike kariuki,100,73,73,True,1.0
46,32255576,32255576,6039357,2023-01-25,velma auma .,nelima auma mbuni,velma auma mbuni,100,67,91,True,1.0
159,36791205,36791205,7070383,2021-12-23,kiptoo bii,evans yegon yegon,kiptoo festus bii,100,19,29,True,1.0
239,28142455,28142455,7359733,2021-12-16,ochieng obunga,paul obunga obunga,ochieng paul obunga,100,71,100,True,1.0
333,26140145,26140145,7461992,2022-11-07,mercy kawira kamau,mecry kawira,mercy kawira kamau,100,73,73,True,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
134078,20875262,20875262,7496234,2022-04-25,salome munyao,salome mbithe none,salome mbithe munyao,100,65,84,True,1.0
134097,26332920,26332920,7396602,2022-05-12,raphael mutai,raphael kipruto none,raphael kipruto mutai,100,70,86,True,1.0
134136,4816559,4816559,7360552,2022-05-23,francis chege ..,francis mwangi none,francis mwangi chege,100,70,85,True,1.0
134195,28181645,28181645,7876389,2022-07-05,juddy maina,juddy nyagaceke none,juddy nyagaceke maina,100,62,86,True,1.0


In [30]:
# Final Dataset
# updates batch 2 - Ops and IPRS names are 70% matched and warehouse and iprs score are not 100 % matched
# updates_batch_2 = final[~final['national_id'].isin(updates_batch_1['national_id'].tolist())]
# updates_batch_2 = updates_batch_2[
#     (updates_batch_2['opt_in_rank'] == 1) &
#     (updates_batch_2['WAREHOUSE-OPS Score'] < 78) &
#     (updates_batch_2['OPS-IPRS Score'] > 70) &
#     (updates_batch_2['WAREHOUSE-IPRS Score'] != 100)
# ].drop_duplicates(subset=['national_id'])
# updates_batch_2
updates_batch_2_ = final[~final['national_id'].isin(updates_batch_1_['national_id'].tolist())]
updates_batch_2_ = updates_batch_2_[
    (updates_batch_2_['opt_in_rank'] == 1) &
    (updates_batch_2_['WAREHOUSE-OPS Score'] < 78) &
    (updates_batch_2_['OPS-IPRS Score'] > 70) &
    (updates_batch_2_['WAREHOUSE-IPRS Score'] != 100)
].drop_duplicates(subset=['national_id'])
updates_batch_2_

,cust_id_nmbr,national_id,mpsa_orga_shrt,opt_in_date,Full Name (OPS),Full Name (WAREHOUSE),Full Name (IPRS),OPS-IPRS Score,WAREHOUSE-OPS Score,WAREHOUSE-IPRS Score,is_iprs_validated,opt_in_rank
1773,30340328,30340328,854312,2022-03-29,erick mueni,acutec entertainment b,eric munene mueni,71,36,46,True,1.0
3991,31309789,31309789,7143931,2022-02-22,nicholus muriuki,douglas mbuthia maina,nicholas mathenge muriuki,73,22,35,True,1.0
5553,29446290,29446290,7653847,2023-03-10,vincent shayo business,vincent s natiri,vincent shayo natiri,79,61,93,True,1.0
6305,12761088,12761088,7334397,2021-11-12,hendry mungai,henry nduati mukami,henry nduati mungai,75,69,89,True,1.0
6753,8059256,8059256,7670335,2021-11-23,scolastica kitivi,theophilus nguta michael,scholastica wanza kitivi,83,29,29,True,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
124465,24679623,24679623,6103136,2023-01-30,margareta mbeneka kivwalu,margaretta m kivwalu,margaretta mbeneka kivwalu,98,76,95,True,1.0
129945,22483992,22483992,7549786,2022-10-31,noam kwamboka,naumi kwamboka makori,naom kwamboka makori,76,76,93,True,1.0
132892,24995505,24995505,7187172,2022-03-17,bernard ongaro,benard ongari none,benard obari ongaro,79,75,76,True,1.0
133475,28582250,28582250,7079224,2022-02-27,maryanne muhoti,mary njoki muthoti,mary njoki muhoti,75,67,97,True,1.0


In [31]:
# OPS is not the same as IPRS and ops national id is same as warehouse national id. Company names and customer names interchanged in OPS list
# problematic_batch = final[
#     (~final['national_id'].isin(updates_batch_1['national_id'].tolist())) &
#     (~final['national_id'].isin(updates_batch_2['national_id'].tolist()))
# ]
# problematic_batch = problematic_batch[
#     (problematic_batch['opt_in_rank'] == 1) &
#     (problematic_batch['WAREHOUSE-IPRS Score'] != 100) &
#     (problematic_batch['cust_id_nmbr'] == problematic_batch['national_id']) &
#     (problematic_batch['OPS-IPRS Score'] < 65)
# ].drop_duplicates(subset=['national_id'])
# problematic_batch
problematic_batch_ = final[
    (~final['national_id'].isin(updates_batch_1_['national_id'].tolist())) &
    (~final['national_id'].isin(updates_batch_2_['national_id'].tolist()))
]
problematic_batch_ = problematic_batch_[
    (problematic_batch_['opt_in_rank'] == 1) &
    (problematic_batch_['WAREHOUSE-IPRS Score'] != 100) &
    (problematic_batch_['cust_id_nmbr'] == problematic_batch_['national_id']) &
    (problematic_batch_['OPS-IPRS Score'] < 65)
].drop_duplicates(subset=['national_id'])
problematic_batch_

,cust_id_nmbr,national_id,mpsa_orga_shrt,opt_in_date,Full Name (OPS),Full Name (WAREHOUSE),Full Name (IPRS),OPS-IPRS Score,WAREHOUSE-OPS Score,WAREHOUSE-IPRS Score,is_iprs_validated,opt_in_rank
14,14690690,14690690,448539,2021-11-23,migrated customer,frider wawira nyaga,stanley kipkemoi koech,26,33,24,True,1.0
144,5327228,5327228,7219203,2022-07-13,maweni community healthy clinic,wali aisha ahmed,wali asha ahmed,30,30,97,True,1.0
180,31910166,31910166,7247976,2022-08-27,believers loveworld church,joan nyakorangi mongare,joan nyakorangi mong'are,24,20,85,True,1.0
230,22043852,22043852,7185732,2022-10-19,delites kitchen,jaquline achieng rabuogi,jaquiline achieng rabuogi,30,31,98,True,1.0
233,23835900,23835900,7317190,2022-10-31,acew kids and dads shop,joseph n ndunyo,joseph njuku ndunyo,29,32,93,True,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
134268,146712,146712,430994,2022-07-12,amicus curiae wines & spirits,lubwa ekron lero,lubwa ekiron lero,27,23,97,True,1.0
134278,26330435,26330435,7543085,2021-11-23,migrated customer,isaac kariru none,isaac kennedy munene kariru,27,29,83,True,1.0
134305,39501361,39501361,7464918,2022-04-22,kennedy kingangi,kenedy kingangi kamuri,kenedy king'ang'i kamuri,55,79,78,True,1.0
134311,34023437,34023437,7667650,2022-07-09,rikab movie store and printers,richard gitonga none,richard kabatha gitonga,30,40,86,True,1.0


In [32]:
# Customers NOT to update
# updates batch 2 - Warehouse and IPRS names are 100% matched and ops national id and warehouse national id are 100% matched
# not_to_update = final[
#     (~final['national_id'].isin(updates_batch_1['national_id'].tolist())) &
#     (~final['national_id'].isin(updates_batch_2['national_id'].tolist())) &
#     (~final['national_id'].isin(problematic_batch['national_id'].tolist()))
# ]
# #
# not_to_update = not_to_update[
#     (not_to_update['opt_in_rank'] == 1) &
#     (not_to_update['WAREHOUSE-OPS Score'] < 78) &
#     (not_to_update['cust_id_nmbr'] == not_to_update['national_id']) &
#     (not_to_update['WAREHOUSE-IPRS Score'] == 100)
# ].drop_duplicates(subset=['national_id'])
#
# not_to_update
not_to_update_ = final[
    (~final['national_id'].isin(updates_batch_1_['national_id'].tolist())) &
    (~final['national_id'].isin(updates_batch_2_['national_id'].tolist())) &
    (~final['national_id'].isin(problematic_batch_['national_id'].tolist()))
]
#
not_to_update_ = not_to_update_[
    (not_to_update_['opt_in_rank'] == 1) &
    (not_to_update_['WAREHOUSE-OPS Score'] < 78) &
    (not_to_update_['cust_id_nmbr'] == not_to_update_['national_id']) &
    (not_to_update_['WAREHOUSE-IPRS Score'] == 100)
].drop_duplicates(subset=['national_id'])

not_to_update_

,cust_id_nmbr,national_id,mpsa_orga_shrt,opt_in_date,Full Name (OPS),Full Name (WAREHOUSE),Full Name (IPRS),OPS-IPRS Score,WAREHOUSE-OPS Score,WAREHOUSE-IPRS Score,is_iprs_validated,opt_in_rank
0,32515103,32515103,170329,2022-12-23,safe space wines and spirit,igantius kimindu maeke,igantius kimindu maeke,33,33,100,True,1.0
1,26637201,26637201,173275,2022-07-26,geoffe mitch services,eric kirimi nkonge,eric kirimi nkonge,36,36,100,True,1.0
4,25509730,25509730,7917688,2022-09-21,obomo bar and restaurant,hellen bosibori nyatome,hellen bosibori nyatome,34,34,100,True,1.0
18,24071799,24071799,5028123,2023-02-09,mawasmac enterprises,margaret njeri waithira,margaret njeri waithira,42,42,100,True,1.0
41,30565442,30565442,528670,2022-11-08,hair by glamourhuose,esther mukami maina,esther mukami maina,31,31,100,True,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
134308,32152780,32152780,7069901,2022-05-03,chepkirib kitchen soup,caren chepkemoi ngetich,caren chepkemoi ngetich,44,44,100,True,1.0
134312,8056005,8056005,7494799,2022-05-14,victoria kamanthe,regina kamuya none,regina none kamuya,57,57,100,True,1.0
134313,6819812,6819812,7618357,2022-05-02,moses kinuthia,emiriana njoki kihagi,emiriana njoki kihagi,34,34,100,True,1.0
134314,28796987,28796987,7854875,2022-05-20,dennis maina macharia,rachael wanjiru nyambura,rachael wanjiru nyambura,40,40,100,True,1.0


In [33]:
# not_to_update2 = final[
#     (~final['national_id'].isin(updates_batch_1['national_id'].tolist())) &
#     (~final['national_id'].isin(updates_batch_2['national_id'].tolist())) &
#     (~final['national_id'].isin(problematic_batch['national_id'].tolist())) &
#     (~final['national_id'].isin(not_to_update['national_id'].tolist()))
# ]
# not_to_update2 = not_to_update2[
#     (not_to_update2['opt_in_rank'] == 1) &
#     (not_to_update2['WAREHOUSE-OPS Score'] == 100) &
#     (not_to_update2['WAREHOUSE-IPRS Score'] == 100) &
#     (not_to_update2['OPS-IPRS Score'] == 100)
# ].drop_duplicates(subset=['national_id'])
# not_to_update2
not_to_update2_ = final[
    (~final['national_id'].isin(updates_batch_1_['national_id'].tolist())) &
    (~final['national_id'].isin(updates_batch_2_['national_id'].tolist())) &
    (~final['national_id'].isin(problematic_batch_['national_id'].tolist())) &
    (~final['national_id'].isin(not_to_update_['national_id'].tolist()))
]
not_to_update2_ = not_to_update2_[
    (not_to_update2_['opt_in_rank'] == 1) &
    (not_to_update2_['WAREHOUSE-OPS Score'] == 100) &
    (not_to_update2_['WAREHOUSE-IPRS Score'] == 100) &
    (not_to_update2_['OPS-IPRS Score'] == 100)
].drop_duplicates(subset=['national_id'])
not_to_update2_

,cust_id_nmbr,national_id,mpsa_orga_shrt,opt_in_date,Full Name (OPS),Full Name (WAREHOUSE),Full Name (IPRS),OPS-IPRS Score,WAREHOUSE-OPS Score,WAREHOUSE-IPRS Score,is_iprs_validated,opt_in_rank
2,23024758,23024758,200725,2022-06-25,jackson wamboo wangechi,jackson wamboo wangechi,jackson wamboo wangechi,100,100,100,True,1.0
5,36209876,36209876,6007413,2022-11-14,kipkorir ken,kipkorir ken bett,kipkorir ken bett,100,100,100,True,1.0
7,32795082,32795082,6008609,2022-10-30,kunte pushati,kunte subeen pushati,kunte subeen pushati,100,100,100,True,1.0
8,20121218,20121218,6009631,2022-11-09,stephen matara nyamichaba,stephen matara nyamichaba,stephen matara nyamichaba,100,100,100,True,1.0
10,30144022,30144022,6011909,2022-11-08,beatrice wairimu nyokabi,beatrice wairimu,beatrice wairimu nyokabi,100,100,100,True,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
134176,33275320,33275320,7580959,2021-11-26,emmanuel makobi,emmanuel wafulah makobi,emmanuel wafulah makobi,100,100,100,True,1.0
134187,32229502,32229502,7543769,2022-12-29,edwin rono,edwin kipyegon rono,edwin kipyegon rono,100,100,100,True,1.0
134206,38995038,38995038,6130230,2023-03-09,mathews otieno rachar,mathews otieno rachar,mathews otieno rachar,100,100,100,True,1.0
134291,31049590,31049590,6123457,2023-03-09,john mwakulomba mwakawi .,john mwakulomba mwakawi,john mwakulomba mwakawi,100,100,100,True,1.0


In [34]:
# not_to_update3 = final[
#     (~final['national_id'].isin(updates_batch_1['national_id'].tolist())) &
#     (~final['national_id'].isin(updates_batch_2['national_id'].tolist())) &
#     (~final['national_id'].isin(problematic_batch['national_id'].tolist())) &
#     (~final['national_id'].isin(not_to_update['national_id'].tolist())) &
#     (~final['national_id'].isin(not_to_update2['national_id'].tolist()))
# ]
# not_to_update3 = not_to_update3[
#     (not_to_update3['opt_in_rank'] == 1) &
#     (not_to_update3['WAREHOUSE-OPS Score'] >= 70) &
#     (not_to_update3['WAREHOUSE-IPRS Score'] >= 70) &
#     (not_to_update3['OPS-IPRS Score'] >= 70)
# ].drop_duplicates(subset=['national_id'])
# not_to_update3
not_to_update3_ = final[
    (~final['national_id'].isin(updates_batch_1_['national_id'].tolist())) &
    (~final['national_id'].isin(updates_batch_2_['national_id'].tolist())) &
    (~final['national_id'].isin(problematic_batch_['national_id'].tolist())) &
    (~final['national_id'].isin(not_to_update_['national_id'].tolist())) &
    (~final['national_id'].isin(not_to_update2_['national_id'].tolist()))
]
not_to_update3_ = not_to_update3_[
    (not_to_update3_['opt_in_rank'] == 1) &
    (not_to_update3_['WAREHOUSE-OPS Score'] >= 70) &
    (not_to_update3_['WAREHOUSE-IPRS Score'] >= 70) &
    (not_to_update3_['OPS-IPRS Score'] >= 70)
].drop_duplicates(subset=['national_id'])
not_to_update3_

,cust_id_nmbr,national_id,mpsa_orga_shrt,opt_in_date,Full Name (OPS),Full Name (WAREHOUSE),Full Name (IPRS),OPS-IPRS Score,WAREHOUSE-OPS Score,WAREHOUSE-IPRS Score,is_iprs_validated,opt_in_rank
21,31376437,31376437,504216,2022-10-19,lilian nyachera nyangaresi,lilian nyanchera nyangaresi,lilian nyanchera nyangaresi,98,98,100,True,1.0
34,33709174,33709174,6037059,2023-03-07,alex musyoka mutua,alex musyoki mutua,alex musyoki mutua,94,94,100,True,1.0
59,31936647,31936647,6076499,2023-02-05,alexina moraa,alexinah moraa osao,alexina moraa osao,100,81,97,True,1.0
62,24106167,24106167,6084596,2023-01-18,alice njeri chege .,alice njeli chege,alice njeri chege,100,94,94,True,1.0
72,32975846,32975846,6105836,2023-01-31,felix kimaru,felex kimaru,felex none kimaru,76,92,100,True,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
134175,24412772,24412772,7885149,2021-12-10,jane yatich,jane yatich none,jane jesang yatich,100,100,81,True,1.0
134185,22010052,22010052,6039255,2023-03-15,antony maina mburu .,anthony maina mburu,anthony maina mburu,97,97,100,True,1.0
134232,25259881,25259881,7317167,2022-05-09,catherine muthoni waitathu,catherine muthoni none,catherine muthoni waitathu,100,87,87,True,1.0
134272,23275594,23275594,7109948,2021-12-19,nancy waruguru ngaru,nancy waruguru none,nancy waruguru ngaru,100,85,85,True,1.0


In [35]:
# not_to_update_4 = final[
#     (~final['national_id'].isin(updates_batch_1['national_id'].tolist())) &
#     (~final['national_id'].isin(updates_batch_2['national_id'].tolist())) &
#     (~final['national_id'].isin(problematic_batch['national_id'].tolist())) &
#     (~final['national_id'].isin(not_to_update['national_id'].tolist())) &
#     (~final['national_id'].isin(not_to_update2['national_id'].tolist())) &
#     (~final['national_id'].isin(not_to_update3['national_id'].tolist()))
# ]
# not_to_update_4 = not_to_update_4[
#     (not_to_update_4['opt_in_rank'] == 1) &
#     (not_to_update_4['WAREHOUSE-OPS Score'] != 100) &
#     (not_to_update_4['OPS-IPRS Score'] == 100)
# ].drop_duplicates(subset=['national_id'])
# not_to_update_4
not_to_update_4_ = final[
    (~final['national_id'].isin(updates_batch_1_['national_id'].tolist())) &
    (~final['national_id'].isin(updates_batch_2_['national_id'].tolist())) &
    (~final['national_id'].isin(problematic_batch_['national_id'].tolist())) &
    (~final['national_id'].isin(not_to_update_['national_id'].tolist())) &
    (~final['national_id'].isin(not_to_update2_['national_id'].tolist())) &
    (~final['national_id'].isin(not_to_update3_['national_id'].tolist()))
]
not_to_update_4_ = not_to_update_4_[
    (not_to_update_4_['opt_in_rank'] == 1) &
    (not_to_update_4_['WAREHOUSE-OPS Score'] != 100) &
    (not_to_update_4_['OPS-IPRS Score'] == 100)
].drop_duplicates(subset=['national_id'])
not_to_update_4_

,cust_id_nmbr,national_id,mpsa_orga_shrt,opt_in_date,Full Name (OPS),Full Name (WAREHOUSE),Full Name (IPRS),OPS-IPRS Score,WAREHOUSE-OPS Score,WAREHOUSE-IPRS Score,is_iprs_validated,opt_in_rank
309,20206179,20206179,7413803,2022-07-01,nicole wangari,nicole wangare none,grace nicole wangari nyambura,100,79,62,True,1.0
2361,21586298,21586298,7473173,2021-11-16,jared mayuya,jerad mayuya mayuya,jared arasa mayuya,100,83,67,True,1.0
13237,26154868,26154868,134748,2021-11-16,eunice adika,eunice anika anika,eunice luvanda adika,100,92,69,True,1.0
18253,30846072,30846072,7231475,2021-11-17,dominic mutua,domic mutua b,dominic kioko mutua,100,85,69,True,1.0
41282,33882721,33882721,7692874,2022-11-01,susan wangari,susan wagari,susan wangari nyaguthii,100,96,69,True,1.0
76671,26616501,26616501,7386079,2021-11-25,willkister wilera,wilkista wilera b,willkister khavakali wilera,100,82,68,True,1.0
76850,28853136,28853136,7122464,2021-12-14,ng'etich elisha,ngetich elisha,ng'etich kipkurui elisha,100,83,63,True,1.0
81673,21804780,21804780,7801935,2022-03-23,rodah njiru,rhoda njiru njiru,rodah wanyaga njiru,100,91,67,True,1.0
98948,24663263,24663263,7272222,2022-09-15,regina oyomba,regina oyumba,regina morine stima oyomba,100,92,63,True,1.0
112490,13524443,13524443,7692676,2022-09-03,florence nkatha,florence karithi,florence nkatha nkumbuku,100,84,67,True,1.0


In [36]:
# updates_batch_3 = final[
#     (~final['national_id'].isin(updates_batch_1['national_id'].tolist())) &
#     (~final['national_id'].isin(updates_batch_2['national_id'].tolist())) &
#     (~final['national_id'].isin(problematic_batch['national_id'].tolist())) &
#     (~final['national_id'].isin(not_to_update['national_id'].tolist())) &
#     (~final['national_id'].isin(not_to_update2['national_id'].tolist())) &
#     (~final['national_id'].isin(not_to_update3['national_id'].tolist())) &
#     (~final['national_id'].isin(not_to_update_4['national_id'].tolist()))
# ]
# consult_collections = updates_batch_3[(updates_batch_3['WAREHOUSE-OPS Score'] < 65) & (~updates_batch_3['national_id'].isin([
#     '26712413', '23309863'
# ]))]
# consult_collections
updates_batch_3_ = final[
    (~final['national_id'].isin(updates_batch_1_['national_id'].tolist())) &
    (~final['national_id'].isin(updates_batch_2_['national_id'].tolist())) &
    (~final['national_id'].isin(problematic_batch_['national_id'].tolist())) &
    (~final['national_id'].isin(not_to_update_['national_id'].tolist())) &
    (~final['national_id'].isin(not_to_update2_['national_id'].tolist())) &
    (~final['national_id'].isin(not_to_update3_['national_id'].tolist())) &
    (~final['national_id'].isin(not_to_update_4_['national_id'].tolist()))
]
consult_collections_ = updates_batch_3_[(updates_batch_3_['WAREHOUSE-OPS Score'] < 65) & (~updates_batch_3_['national_id'].isin([
    '26712413', '23309863'
]))]
consult_collections_

,cust_id_nmbr,national_id,mpsa_orga_shrt,opt_in_date,Full Name (OPS),Full Name (WAREHOUSE),Full Name (IPRS),OPS-IPRS Score,WAREHOUSE-OPS Score,WAREHOUSE-IPRS Score,is_iprs_validated,opt_in_rank
156,36797129,36797129,7441554,2022-10-10,airlift fortune,felex muriuki muriithi,felex muriuki muriithi,32,32,100,True,2.0
227,35021307,35021307,7690708,2022-08-02,wallet exchanger services hq,ronald ashioyamuleshe odangah,ronald ashioya muleshe odangah,34,35,98,True,2.0
597,24455442,24455442,7071456,2021-11-15,migrated customer,anthony weru muriuki,anthony weru muriuki,32,32,100,True,2.0
771,23013938,23013938,271581,2022-05-19,economy general,jefferson nyange mwinzi,jefferson nyange mwinzi,37,37,100,True,2.0
2089,21603768,21603768,511241,2022-06-07,meaty meat and veg corner,stephen mutuku musembi,stephen mutuku musembi,30,30,100,True,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...
123599,20056932,20056932,7687645,2022-07-25,expression barber shop,patrick thiongo maina,patrick thiongo maina,37,37,100,True,2.0
124426,28882535,28882535,876331,2022-09-15,home choice self,ismael morogo,ismael kiprotich morogo,26,28,100,True,2.0
125639,35304660,35304660,7010634,2022-10-08,amok fit,joe derrick odhiambo onyango,joe derrick odhiambo onyango,22,22,100,True,2.0
133563,6559829,6559829,514844,2022-06-28,evans ombati.,mellen kerubo none,mellen kerubo ombati,67,27,84,True,1.0


In [38]:
final_updates = pd.concat([updates_batch_1_, updates_batch_2_]).drop(columns=['opt_in_rank', 'is_iprs_validated'])
final_updates.to_csv(f'~/final_kyc_updates_{datetime.datetime.now()}_batch2.csv', index=False)

In [48]:
final[final['mpsa_orga_shrt'] == 279851]

,cust_id_nmbr,national_id,mpsa_orga_shrt,opt_in_date,Full Name (OPS),Full Name (WAREHOUSE),Full Name (IPRS),OPS-IPRS Score,WAREHOUSE-OPS Score,WAREHOUSE-IPRS Score,is_iprs_validated,opt_in_rank
6548,20411464,20411464,279851,2021-11-12,carolyne wambui kagwi,carolyne wambui kagwi,carolyne wambui kagwi,100,100,100,True,2.0


In [21]:
final_updates

NameError: name 'final_updates' is not defined

In [5]:
updated = pd.read_csv('/home/henrykuria/final_kyc_updates_2023-04-24 11:32:58.681361_batch2.csv')
updates = []
updated.head()
for i, r in updated.iterrows():
    full_name = r['Full Name (IPRS)'].split()  # Split full name by space
    first_name = full_name[0].replace("'", "").capitalize() if len(full_name) > 0 else None  # Extract first name
    middle_name = full_name[1].replace("'", "").capitalize() if len(full_name) > 1 else None  # Extract middle name
    last_name = ' '.join(full_name[2:]).replace("'", "").capitalize() if len(full_name) > 2 else None  # Concatenate remaining names as last name

    # Build the SQL update statement
    update = f"update bloomlive.client_dimension set first_name='{first_name}', middle_name='{middle_name}', last_name='{last_name}' where national_id='{r['national_id']}' and bloom_version='2'"
    updates.append(update)

# print(updates)
# with open('/home/henrykuria/data/safaricom_bloom/KYC_customer_name_updates_2023-04-24.csv', 'w') as f:
#     for update in updates:
#         f.write(f'{update}\n')
#
warehouse_hook.run(sql=updates + [
    "update bloomlive.client_dimension set first_name = trim(first_name)",
    "update bloomlive.client_dimension set middle_name = trim(middle_name)",
    "update bloomlive.client_dimension set last_name = trim(last_name)",
    "update bloomlive.client_dimension set first_name = null where first_name = 'None'",
    "update bloomlive.client_dimension set middle_name = null where middle_name = 'None'",
    "update bloomlive.client_dimension set last_name = null where last_name = 'None'"
])
# updates.to_csv('KYC_customer_names_2023-04-24.csv')

[2023-06-22 12:58:37,341] {base.py:73} INFO - Using connection ID 'rds_afsg_ds_prod_postgresql_dwh' for task execution.
[2023-06-22 12:58:38,688] {sql.py:364} INFO - Running statement: update bloomlive.client_dimension set first_name='Jasan', middle_name='Nyoike', last_name='Kariuki' where national_id='8633566' and bloom_version='2', parameters: None
[2023-06-22 12:58:38,999] {sql.py:373} INFO - Rows affected: 1
[2023-06-22 12:58:39,000] {sql.py:364} INFO - Running statement: update bloomlive.client_dimension set first_name='Velma', middle_name='Auma', last_name='Mbuni' where national_id='32255576' and bloom_version='2', parameters: None
[2023-06-22 12:58:39,164] {sql.py:373} INFO - Rows affected: 1
[2023-06-22 12:58:39,165] {sql.py:364} INFO - Running statement: update bloomlive.client_dimension set first_name='Kiptoo', middle_name='Festus', last_name='Bii' where national_id='36791205' and bloom_version='2', parameters: None
[2023-06-22 12:58:39,328] {sql.py:373} INFO - Rows affected:

In [1]:
updated = pd.read_csv('/home/henrykuria/final_kyc_updates_2023-04-24 11:32:58.681361_batch2.csv')

NameError: name 'pd' is not defined

In [20]:
updated[updated['mpsa_orga_shrt'] == 279851]

,cust_id_nmbr,national_id,mpsa_orga_shrt,opt_in_date,Full Name (OPS),Full Name (WAREHOUSE),Full Name (IPRS),OPS-IPRS Score,WAREHOUSE-OPS Score,WAREHOUSE-IPRS Score


In [44]:
not_to_update_4_[not_to_update_4_['mpsa_orga_shrt'] == 279851]

,cust_id_nmbr,national_id,mpsa_orga_shrt,opt_in_date,Full Name (OPS),Full Name (WAREHOUSE),Full Name (IPRS),OPS-IPRS Score,WAREHOUSE-OPS Score,WAREHOUSE-IPRS Score,is_iprs_validated,opt_in_rank


In [82]:
ops_list.shape

(110040, 6)

In [36]:
consult_collections.to_csv('~/consult_collections.csv', index=False)

In [40]:
mifos_ids = warehouse_hook.get_pandas_df(
    sql="""select mifos_id from bloomlive.client_materialized_summary_view where national_id in %(ni)s and bloom_version=2""",
    parameters={'ni': tuple(final_updates['national_id'].to_list())}
)

[2023-04-19 09:53:03,156] {base.py:73} INFO - Using connection ID 'airflow_warehouse_user' for task execution.


/home/henrykuria/airflow_venv/lib/python3.10/site-packages/pandas/io/sql.py:761 UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy

In [43]:
mifos_clients = mifos_hook.get_pandas_df(
    sql="""select * from `mifostenant-safaricom`.m_client where external_id in %(mi)s""",
    parameters={'mi': tuple(mifos_ids['mifos_id'].tolist())}
)

[2023-04-19 09:57:26,542] {base.py:73} INFO - Using connection ID 'mifos_db' for task execution.


/home/henrykuria/airflow_venv/lib/python3.10/site-packages/pandas/io/sql.py:761 UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy

In [44]:
mifos_clients

,id,account_no,external_id,status_enum,sub_status,activation_date,office_joining_date,office_id,transfer_to_office_id,staff_id,...,rejectedon_date,rejectedon_userid,withdraw_reason_cv_id,withdrawn_on_date,withdraw_on_userid,reactivated_on_date,reactivated_on_userid,legal_form_enum,reopened_on_date,reopened_by_userid


In [ ]:
# OPS is not the same as IPRS and ops national id is same as warehouse national id
final[
    (final['opt_in_rank'] == 1) &
    (final['WAREHOUSE-IPRS Score'] != 100) &
    (final['cust_id_nmbr'] == final['national_id']) &
    (final['OPS-IPRS Score'] < 65)
].drop_duplicates(subset=['national_id'])

In [ ]:
# Need to repeat iprs
# OPS is not the same as IPRS and ops national id is same as warehouse national id and iprs like None
redo_iprs = final[
    (final['opt_in_rank'] == 1) &
    (final['OPS-IPRS Score'] != 100) &
    (final['cust_id_nmbr'] == final['national_id']) &
    (final['Full Name (IPRS)'] == 'None None None')
].drop_duplicates(subset=['national_id'])

In [ ]:
# redo_iprs[redo_iprs['national_id'] == '10100320']
redo_iprs.shape

In [ ]:
merged[merged.index == 517][['ds_customer_name', 'ds_company_name']]

### Visualizations of Fuzzy Match Customer Names

In [ ]:
cutoff_score = 50
sns.displot(final, x='WAREHOUSE-OPS Score', kde=True)
plt.axvline(x=cutoff_score, color='r', linestyle='--')
plt.show()

In [ ]:
cutoff_score = 70
sns.scatterplot(data=final, x='WAREHOUSE-OPS Score', y='WAREHOUSE-IPRS Score', hue='Full Name (WAREHOUSE)', palette='bright')
plt.axhline(y=cutoff_score, color='r', linestyle='--')
plt.axvline(x=cutoff_score, color='r', linestyle='--')
plt.show()

,time_,credit_amt,debit_amt,terminal,biller,narration,balance,trxn_type,status,request_ref
0,2022-10-15 23:59,0.0,48186774.0,3IS11633,opening debit sum,NaN,-48179156.0,Payment,Approved,DRRECON-08-16-18241
1,2022-10-15 23:59,48193676.0,0.0,3IS11633,opening credit sum,NaN,14520.0,Payment,Approved,RECON-CR-20220816-3IS11633
2,2023-04-14 16:22,0.0,1000.0,3IS11633,MTN Airtime,NaN,13520.0,Payment,Approved,ISWM16814785304931633
3,2023-04-15 14:10,0.0,5000.0,3IS11633,"MTN Data freedom1.5GB for 5,000",NaN,8520.0,Payment,Approved,ISWM16815570460371633
4,2023-04-17 11:01,0.0,900.0,3IS11633,MTN Airtime,NaN,7620.0,Payment,Approved,ISWM16817184676521633
5,2023-04-18 13:31,44000.0,0.0,3IS11633,MTN WITHDRAW,NaN,51620.0,Payment,Approved,CW-ISWM16818138522601633
6,2023-04-18 14:07,0.0,10000.0,3IS11633,MTN DEPOSIT,NaN,41620.0,Payment,Approved,ISWM16818160700551633
7,2022-10-15 23:59,48193676.0,0.0,3IS11633,opening credit sum,NaN,48201294.0,Payment,Approved,RECON-CR-20220816-3IS11633
8,2022-10-15 23:59,0.0,48186774.0,3IS11633,opening debit sum,NaN,14520.0,Payment,Approved,DRRECON-08-16-18241
